In [1]:
import pandas as pd

In [2]:
# read in original data.  this data contains cumulative cases per texas county per day
# a little extra provisions necessary to skip extraneous rows and to make columns what we want them to be

data_file_df = pd.read_excel('Texas COVID-19 Case Count Data by County.xlsx', skiprows={0, 1, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267})
data_file_df.rename (columns=lambda x: x[-4:], inplace=True)   # This will need adjustment come October

# data for 3-07 and 3-08 are missing - so we'll pretend that any data before that is missing as well.
# these are mostly zero anyway.
del data_file_df["3-04"]
del data_file_df["3-05"]
del data_file_df["3-06"]

# a few more fixes we know we need to make:
# data_file_df.rename(columns={'-15*':'7-15', '17**':'7-17'}  )

# the above rename didn't work, so here is an alternative (and kludgy) way of doing it:
cols = data_file_df.columns.values
x = 0
for col in cols:
    if col == '-15*':
        cols[x] = '7-15'
    elif col == '17**':
        cols[x] = '7-17'
    x = x + 1
    
data_file_df.columns = cols

In [3]:
def assign_grade(value):

    
# original binning values:   20  10  8  6  4  2  >0  0  <0
#          labels:           A   B   C  D  E  F  G       N

    grade = " "
    if value >= 40:
        grade = ">40"
    elif value >= 28:
        grade = ">28"
    elif value >= 21:
        grade = ">21"
    elif value >= 15:
        grade = ">15"
    elif value >= 10:
        grade = ">10"
    elif value >= 5:
        grade = ">5"
    elif value > 0:
        grade = ">0"
    elif value == 0:
        grade = "none"
    elif value < 0:
        grade = "negative"
        
    return (grade);


In [4]:
# Additional county information not present in main data file

codes_df = pd.read_csv('codes2.csv')
codes_df.head()

,Name,Section,Number,Population,FIPS
0,Anderson,J,11,58199,48001
1,Andrews,C,11,22269,48003
2,Angelina,J,23,90437,48005
3,Aransas,N,29,27699,48007
4,Archer,F,13,8344,48009


In [5]:
covid19_df = pd.merge(codes_df, data_file_df,  on="Name")
covid19_df

,Name,Section,Number,Population,FIPS,3-09,3-10,3-11,3-12,3-13,...,7-27,7-28,7-29,7-30,7-31,8-01,8-02,8-03,8-04,8-05
0,Anderson,J,11,58199,48001,0,0,0,0,0,...,2142,2162,2172,2174,2196,2209,2209,2307,2329,2355
1,Andrews,C,11,22269,48003,0,0,0,0,0,...,228,232,238,253,255,255,266,267,268,273
2,Angelina,J,23,90437,48005,0,0,0,0,0,...,1405,1487,1566,1606,1606,1647,1647,1647,1745,1755
3,Aransas,N,29,27699,48007,0,0,0,0,0,...,131,135,136,149,153,153,157,158,158,162
4,Archer,F,13,8344,48009,0,0,0,0,0,...,15,15,15,17,17,20,20,20,20,20
5,Armstrong,E,33,1948,48011,0,0,0,0,0,...,4,5,5,5,5,8,8,8,9,7
6,Atascosa,N,14,51831,48013,0,0,0,0,0,...,393,396,396,405,405,405,405,405,413,419
7,Austin,K,12,30402,48015,0,0,0,0,0,...,193,202,209,211,212,213,213,213,216,218
8,Bailey,D,11,7692,48017,0,0,0,0,0,...,132,141,148,151,156,158,158,158,159,166
9,Bandera,B,25,21246,48019,0,0,0,0,0,...,71,78,79,83,83,83,83,83,83,89


In [6]:
# create a parallel dataframe with rates per 10000 people in county
covid19rate_df = covid19_df.copy()

ylim, xlim = covid19rate_df.shape

for y in range(0, ylim):
    pop10000 = covid19rate_df.iloc[y,3]/10000
    for x in range(5, xlim):
        covid19rate_df.iloc[y,x] = covid19rate_df.iloc[y,x]/pop10000

In [7]:
 covid19rate_df

,Name,Section,Number,Population,FIPS,3-09,3-10,3-11,3-12,3-13,...,7-27,7-28,7-29,7-30,7-31,8-01,8-02,8-03,8-04,8-05
0,Anderson,J,11,58199,48001,0.0,0.0,0.0,0.0,0.0,...,368.047561,371.484046,373.202289,373.545937,377.326071,379.559786,379.559786,396.398564,400.178697,404.646128
1,Andrews,C,11,22269,48003,0.0,0.0,0.0,0.0,0.0,...,102.384481,104.180700,106.875028,113.610849,114.508959,114.508959,119.448561,119.897616,120.346670,122.591944
2,Angelina,J,23,90437,48005,0.0,0.0,0.0,0.0,0.0,...,155.356768,164.423853,173.159216,177.582184,177.582184,182.115727,182.115727,182.115727,192.952000,194.057742
3,Aransas,N,29,27699,48007,0.0,0.0,0.0,0.0,0.0,...,47.294126,48.738222,49.099245,53.792556,55.236651,55.236651,56.680747,57.041770,57.041770,58.485866
4,Archer,F,13,8344,48009,0.0,0.0,0.0,0.0,0.0,...,17.976989,17.976989,17.976989,20.373921,20.373921,23.969319,23.969319,23.969319,23.969319,23.969319
5,Armstrong,E,33,1948,48011,0.0,0.0,0.0,0.0,0.0,...,20.533881,25.667351,25.667351,25.667351,25.667351,41.067762,41.067762,41.067762,46.201232,35.934292
6,Atascosa,N,14,51831,48013,0.0,0.0,0.0,0.0,0.0,...,75.823349,76.402153,76.402153,78.138566,78.138566,78.138566,78.138566,78.138566,79.682044,80.839652
7,Austin,K,12,30402,48015,0.0,0.0,0.0,0.0,0.0,...,63.482666,66.442997,68.745477,69.403329,69.732254,70.061180,70.061180,70.061180,71.047957,71.705809
8,Bailey,D,11,7692,48017,0.0,0.0,0.0,0.0,0.0,...,171.606864,183.307332,192.407696,196.307852,202.808112,205.408216,205.408216,205.408216,206.708268,215.808632
9,Bandera,B,25,21246,48019,0.0,0.0,0.0,0.0,0.0,...,33.418055,36.712793,37.183470,39.066177,39.066177,39.066177,39.066177,39.066177,39.066177,41.890238


In [8]:
# create a parallel dataframe showing each day's increment of cases per county
covid19incr_df = covid19_df.copy()
ylim, xlim = covid19incr_df.shape

for y in range(0, ylim):
    for x in range(6, xlim):
        covid19incr_df.iloc[y,x] = covid19_df.iloc[y,x] - covid19_df.iloc[y,x-1]

In [9]:
# compute the seven-day rolling average of those increments

covid19ravg_df = covid19_df.copy()
ylim, xlim = covid19ravg_df.shape

for y in range(0, ylim):
    pop100000 = covid19rate_df.iloc[y,3]/100000
    for x in range(11, xlim):
        covid19ravg_df.iloc[y,x] = ((covid19incr_df.iloc[y,x] + covid19incr_df.iloc[y,x-1] +
                                    covid19incr_df.iloc[y,x-2] + covid19incr_df.iloc[y,x-3] +
                                    covid19incr_df.iloc[y,x-4] + covid19incr_df.iloc[y,x-5] +
                                    covid19incr_df.iloc[y,x-6]) /  7) / pop100000

In [10]:
# these days come before a seven-day rolling average can be computed, and so are not wanted

del covid19ravg_df["3-09"]
del covid19ravg_df["3-10"]
del covid19ravg_df["3-11"]
del covid19ravg_df["3-12"]
del covid19ravg_df["3-13"]
del covid19ravg_df["3-15"]  # 3-14 is also missing.

covid19ravg_df

,Name,Section,Number,Population,FIPS,3-16,3-17,3-18,3-19,3-20,...,7-27,7-28,7-29,7-30,7-31,8-01,8-02,8-03,8-04,8-05
0,Anderson,J,11,58199,48001,0.000000,0.000000,0.000000,0.000000,0.000000,...,31.173830,27.246418,27.000955,23.809933,19.637058,16.936963,16.936963,40.501432,40.992359,44.919770
1,Andrews,C,11,22269,48003,0.000000,0.000000,0.000000,0.000000,0.000000,...,25.018764,21.811230,16.679176,24.377257,23.094244,19.245203,26.301778,25.018764,23.094244,22.452737
2,Angelina,J,23,90437,48005,0.000000,0.000000,0.000000,0.000000,0.000000,...,63.975080,76.928059,52.443769,58.762295,35.225785,38.227085,38.227085,38.227085,40.754495,29.855037
3,Aransas,N,29,27699,48007,0.000000,0.000000,0.000000,0.000000,0.000000,...,20.114187,14.956703,12.893709,18.051193,19.082690,15.472451,15.988200,13.925206,11.862213,13.409458
4,Archer,F,13,8344,48009,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.136283,3.424188,3.424188,6.848377,6.848377,8.560471,8.560471,8.560471,8.560471,8.560471
5,Armstrong,E,33,1948,48011,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.333529,14.667058,14.667058,14.667058,7.333529,29.334116,29.334116,29.334116,29.334116,14.667058
6,Atascosa,N,14,51831,48013,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.473600,7.166147,4.685558,4.134316,3.858695,3.307453,3.307453,3.307453,4.685558,6.339284
7,Austin,K,12,30402,48015,0.000000,0.000000,0.000000,0.000000,0.000000,...,15.506499,17.386074,18.795756,19.265650,19.735544,15.036605,14.566711,9.397878,6.578515,4.229045
8,Bailey,D,11,7692,48017,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.000520,27.858257,40.858777,42.715994,48.287646,48.287646,48.287646,48.287646,33.429909,33.429909
9,Bandera,B,25,21246,48019,0.000000,0.000000,0.000000,0.000000,0.000000,...,15.465096,18.154678,16.809887,18.154678,11.430723,8.068746,8.068746,8.068746,3.361977,6.723955


In [11]:
# assign grades to rates of occurance

covid19grad_df = covid19ravg_df.copy()
ylim, xlim = covid19grad_df.shape

for y in range(0, ylim):
    for x in range(5, xlim):
#        if (covid19grad_df.iloc[y,x] >= 20):
#            covid19grad_df.iloc[y,x] = "A"
#        elif (covid19grad_df.iloc[y,x] >= 10):
#            covid19grad_df.iloc[y,x] = "B"
#        elif (covid19grad_df.iloc[y,x] >= 8):
#            covid19grad_df.iloc[y,x] = "C"
#        elif (covid19grad_df.iloc[y,x] >= 6):
#            covid19grad_df.iloc[y,x] = "D"
#        elif (covid19grad_df.iloc[y,x] >= 4):
#            covid19grad_df.iloc[y,x] = "E"
#        elif (covid19grad_df.iloc[y,x] >= 2):
#            covid19grad_df.iloc[y,x] = "F"
#        elif (covid19grad_df.iloc[y,x] > 0):
#            covid19grad_df.iloc[y,x] = "G";
#        else: covid19grad_df.iloc[y,x] = " ";
        covid19grad_df.iloc[y,x] = assign_grade(covid19grad_df.iloc[y,x])
            
covid19grad_df
            

,Name,Section,Number,Population,FIPS,3-16,3-17,3-18,3-19,3-20,...,7-27,7-28,7-29,7-30,7-31,8-01,8-02,8-03,8-04,8-05
0,Anderson,J,11,58199,48001,none,none,none,none,none,...,>28,>21,>21,>21,>15,>15,>15,>40,>40,>40
1,Andrews,C,11,22269,48003,none,none,none,none,none,...,>21,>21,>15,>21,>21,>15,>21,>21,>21,>21
2,Angelina,J,23,90437,48005,none,none,none,none,none,...,>40,>40,>40,>40,>28,>28,>28,>28,>40,>28
3,Aransas,N,29,27699,48007,none,none,none,none,none,...,>15,>10,>10,>15,>15,>15,>15,>10,>10,>10
4,Archer,F,13,8344,48009,none,none,none,none,none,...,>5,>0,>0,>5,>5,>5,>5,>5,>5,>5
5,Armstrong,E,33,1948,48011,none,none,none,none,none,...,>5,>10,>10,>10,>5,>28,>28,>28,>28,>10
6,Atascosa,N,14,51831,48013,none,none,none,none,none,...,>10,>5,>0,>0,>0,>0,>0,>0,>0,>5
7,Austin,K,12,30402,48015,none,none,none,none,none,...,>15,>15,>15,>15,>15,>15,>10,>5,>5,>0
8,Bailey,D,11,7692,48017,none,none,none,none,none,...,>10,>21,>40,>40,>40,>40,>40,>40,>28,>28
9,Bandera,B,25,21246,48019,none,none,none,none,none,...,>15,>15,>15,>15,>10,>5,>5,>5,>0,>5


In [12]:
covid19rate_df.to_csv("covid19rate.csv", index=False, header=True)
covid19ravg_df.to_csv("covid19ravg.csv", index=False, header=True)
covid19grad_df.to_csv("covid19grad.csv", index=False, header=True)
covid19incr_df.to_csv("covid19incr.csv", index=False, header=True)

In [13]:
# This puts the data in a format that's friendlier to Tableau

counties = []
populations = []
dates = []
values = []
fips = []
grades = []
columns = covid19ravg_df.columns

ylim, xlim = covid19ravg_df.shape

for y in range(0, ylim):
    cty = covid19ravg_df.iloc[y,0]
    pop = covid19ravg_df.iloc[y,3]
    fip = covid19ravg_df.iloc[y,4]
    for x in range(5, xlim):
        counties.append(cty)
        populations.append(pop)
        fips.append(fip)
        dates.append(columns[x])
        values.append(covid19ravg_df.iloc[y,x])
        grades.append(assign_grade(covid19ravg_df.iloc[y,x]))
        
covid19ravg2_df = pd.DataFrame({
    "Counties": counties,
    "Populations": populations,
    "FIPS": fips,
    "Dates": dates,
    "Values": values,
    "Grades": grades
    })

covid19ravg2_df

,Counties,Populations,FIPS,Dates,Values,Grades
0,Anderson,58199,48001,3-16,0.000000,none
1,Anderson,58199,48001,3-17,0.000000,none
2,Anderson,58199,48001,3-18,0.000000,none
3,Anderson,58199,48001,3-19,0.000000,none
4,Anderson,58199,48001,3-20,0.000000,none
5,Anderson,58199,48001,3-21,0.000000,none
6,Anderson,58199,48001,3-22,0.000000,none
7,Anderson,58199,48001,3-23,0.000000,none
8,Anderson,58199,48001,3-24,0.000000,none
9,Anderson,58199,48001,3-25,0.000000,none


In [14]:
covid19ravg2_df.to_csv("covid19ravg2.csv", index=False, header=True)